In [1]:
import pandas as pd

# Load the data
df = pd.read_csv("w11/PracticalLab/data/Forbes2000.csv", encoding='ISO-8859-1', index_col=0)

# Check the structure
print(df.columns)
df.head()

Index(['rank', 'name', 'country', 'category', 'sales', 'profits', 'assets',
       'marketvalue'],
      dtype='object')


,rank,name,country,category,sales,profits,assets,marketvalue
1,1,Citigroup,United States,Banking,94.71,17.85,1264.03,255.30
2,2,General Electric,United States,Conglomerates,134.19,15.59,626.93,328.54
3,3,American Intl Group,United States,Insurance,76.66,6.46,647.66,194.87
4,4,ExxonMobil,United States,Oil & gas operations,222.88,20.96,166.99,277.02
5,5,BP,United Kingdom,Oil & gas operations,232.57,10.27,177.57,173.54


In [2]:
df_clean_cat = df[['sales', 'profits', 'assets', 'marketvalue', 'category']].dropna()


In [3]:
df_clean_cat['category'].value_counts()


category
Banking                             313
Diversified financials              158
Insurance                           110
Utilities                           110
Materials                            97
Oil & gas operations                 90
Retailing                            88
Food drink & tobacco                 83
Construction                         79
Transportation                       78
Consumer durables                    74
Business services & supplies         70
Telecommunications services          66
Health care equipment & services     65
Media                                61
Technology hardware & equipment      59
Capital goods                        53
Chemicals                            50
Drugs & biotechnology                45
Household & personal products        44
Hotels restaurants & leisure         37
Food markets                         33
Conglomerates                        31
Software & services                  31
Semiconductors                 

In [4]:
df_dummies = pd.get_dummies(df_clean_cat, columns=['category'], drop_first=True)


We omitted the "Financials" category because it was the most common and serves as a neutral baseline for comparison.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Define X and y
X = df_dummies.drop('marketvalue', axis=1)
y = df_dummies['marketvalue']

# Fit the model
model = LinearRegression().fit(X, y)
y_pred = model.predict(X)

# Output metrics
print("=== Scikit-Learn Multiple Regression with Categorical Variable ===")
print(f"Intercept: {model.intercept_:.4f}")
print(f"R²: {r2_score(y, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y, y_pred)):.4f}")


=== Scikit-Learn Multiple Regression with Categorical Variable ===
Intercept: 1.0794
R²: 0.6154
RMSE: 15.1817


In [7]:
import statsmodels.api as sm

# Add intercept and ensure numeric dtype
X_sm = sm.add_constant(X)
X_sm = X_sm.astype(float)

# Fit the model
model_sm = sm.OLS(y, X_sm).fit()
print("=== StatsModels Summary ===")
print(model_sm.summary())


=== StatsModels Summary ===
                            OLS Regression Results                            
Dep. Variable:            marketvalue   R-squared:                       0.615
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     108.4
Date:                Mon, 07 Apr 2025   Prob (F-statistic):               0.00
Time:                        17:13:28   Log-Likelihood:                -8257.4
No. Observations:                1995   AIC:                         1.657e+04
Df Residuals:                    1965   BIC:                         1.674e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

### 🧾 PE07-1-3 — Multiple Linear Regression with Categorical Variable

In this part, we extended our multiple linear regression model by adding the **categorical variable `category`**, alongside the numerical variables `sales`, `profits`, and `assets`.

---

#### 📘 Dummy Encoding

We used **one-hot encoding** for `category` and **omitted the `Banking` category** to avoid multicollinearity (dummy variable trap). `"Banking"` was the most frequent category in the dataset and serves as a neutral baseline.

---

#### 📊 Scikit-Learn Results

| Metric           | Value     |
|------------------|-----------|
| **Intercept**     | 1.0794    |
| **R²**            | **0.6154** |
| **RMSE**          | **15.1817** |

---

#### 📘 StatsModels Results – Key Highlights

- **R²**: 0.615 → 61.5% of variance in `marketvalue` is explained by the model  
- **All numeric predictors (`sales`, `profits`, `assets`) remain statistically significant** (p < 0.001)  
- **Some categories**, like:
  - `Drugs & biotechnology` (β = 25.43, **p < 0.001**),
  - `Semiconductors` (β = 16.86, **p < 0.001**),
  - `Software & services` (β = 15.34, **p < 0.01**),
  - `Telecommunications services` (β = 15.69, **p < 0.001**)  
  contribute significantly to increasing `marketvalue`.

- A few categories (like `Trading companies`, β = -16.66, **p < 0.001**) are significantly **associated with lower market values**.

---

### 🔍 Comparison with PE07-1-2 (No Categorical Variable)

| Model                              | R²     | RMSE     |
|-----------------------------------|--------|----------|
| **PE07-1-2**: w/o category         | 0.5435 | 16.5406  |
| **PE07-1-3**: with category        | **0.6154** | **15.1817** |

---

### ✅ Final Conclusion

> **Including `category` improves the regression model’s performance significantly.**  
> The R² increased from **0.5435 → 0.6154**, and RMSE dropped from **16.54 → 15.18**.  
> This shows that a company's industry **category provides meaningful predictive power** when estimating its market value.

The categorical variables enhance the model's explanatory strength by capturing sector-based variations that numeric financials alone cannot explain.

